In [1]:
import time
import ROOT
import numpy as np
import pandas as pd
import root_pandas as rpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from root_pandas import read_root
from numpy import inf
from matplotlib import rc
rc('text', usetex=True)

Welcome to JupyROOT 6.15/01


In [2]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut)
    return cutDataframe

In [3]:
def getJetData(inputFiles):
    
    cols = []
    scalar = []
    scalar.append('evid')
    scalar.append('xsec')
    scalar.append('ntrials')    
    scalar.append('x')
    scalar.append('y')
    scalar.append('Q2')
    scalar.append('W2')
    scalar.append('e_eta')
    scalar.append('e_theta')
    scalar.append('e_p')
    scalar.append('e_pt')

    cols.append('n_total')
    cols.append('jet_eta')
    cols.append('jet_theta')
    cols.append('jet_p')
    cols.append('jet_pt')
    cols.append('dphi_e_jet')
    cols.append('dR_q_jet')
    
    lists = scalar
    dataframes = []
    for inputFilename in inputFiles:
        start = time.time()
        df = read_root(inputFilename, columns=lists+cols,flatten=cols)
        dataframes.append(df)
        end = time.time()
        print '\n', 'Processed', inputFilename, 'in',  '%2.1f'%(end-start), 'seconds'
    return pd.concat([x for x in dataframes])

In [4]:
def getLeptonData(inputFiles):
    cols = []
    scalar = []
    scalar.append('evid')
    scalar.append('xsec')
    scalar.append('ntrials')    
    scalar.append('x')
    scalar.append('y')
    scalar.append('Q2')
    scalar.append('W2')
    scalar.append('e_eta')
    scalar.append('e_theta')
    scalar.append('e_p')
    scalar.append('e_pt')

    
    lists = scalar
    dataframes = []
    for inputFilename in inputFiles:
        start = time.time()
        df = read_root(inputFilename, columns=lists+cols,flatten=cols)
        dataframes.append(df)
        end = time.time()
        print '\n', 'Processed', inputFilename, 'in',  '%2.1f'%(end-start), 'seconds'
    return pd.concat([x for x in dataframes])

In [11]:
##Selection on event variables
def applyEventCuts(df):
    print '################### ' 
    print 'Applying event selection'
    temp = df 
    temp = applyCut(temp, 'y < 0.85 and y>0.1', '0.1 < y < 0.85')
    temp = applyCut(temp, 'Q2>100', 'Q2>100')
    temp.eval('logQ2= log(Q2)/2.3025850', inplace=True)
    temp.eval('logx= log(x)/2.3025850', inplace=True)
    print '################### \n' 
    return temp

In [18]:
## Selection on event and jet variables
def applyJetCuts(df):
    print '################### ' 
    print 'Applying jet selection'
    temp = df
    temp = applyCut(temp, 'jet_pt>5', ' jet pT > 5 GeV') 
    print '################### \n' 
    temp.eval('jet_qtnorm= jet_qt/sqrt(Q2)', inplace=True)
    temp.eval('jet_qtnormjetpt= jet_qt/sqrt(jet_pt)', inplace=True)
    temp.eval('jet_qtnormept= jet_qt/sqrt(e_pt)', inplace=True)

    return temp

In [7]:
def printSummary(df):
    print df['e_pt'].mean(), ' mean lepton pt'
    print df['y'].mean(), ' mean inelasticity'
    print df['jet_p'].mean(), ' mean jet p'
    print df['jet_pt'].mean(), ' mean jet pT'
    print df['jet_pt'].describe()

    print df['x'].mean(), ' mean x'
    print df['Q2'].mean(), ' mean Q2'
    print df['Q2'].describe()
    return

## Get neutral current data

In [15]:
df_inclusive = {}
##Neutral current
df_inclusive['NC'] = getLeptonData(['NC_10_275.root'])
df_inclusive['NC'] = applyEventCuts(df_inclusive['NC'])
## Charged current
df_inclusive['CC'] = getLeptonData(['CC_10_275.root'])
df_inclusive['CC'] = applyEventCuts(df_inclusive['CC'])


Processed NC_10_275.root in 0.2 seconds

Processed CC_10_275.root in 0.0 seconds
################### 
Applying event selection
0.1 < y < 0.85 133361  fraction kept: 53.3
Q2>100 25028  fraction kept: 18.8
################### 

################### 
Applying event selection
0.1 < y < 0.85 7976  fraction kept: 79.8
Q2>100 7976  fraction kept: 100.0
################### 



## Get jet data

In [22]:
df_jet = {}
df_jet['NC'] = getJetData(['NC_10_275.root'])
df_jet['CC'] = getJetData(['CC_10_275.root'])
df_jet['NC'] = applyEventCuts(df_jet['NC'])
df_jet['CC'] = applyEventCuts(df_jet['CC'])
df_jet['NC'] = applyJetCuts(df_jet['NC'])
df_jet['CC'] = applyJetCuts(df_jet['CC'])


Processed NC_10_275.root in 5.6 seconds

Processed CC_10_275.root in 0.2 seconds
################### 
Applying event selection
0.1 < y < 0.85 127032  fraction kept: 47.1
Q2>100 40158  fraction kept: 31.6
################### 

################### 
Applying event selection
0.1 < y < 0.85 14226  fraction kept: 79.8
Q2>100 14226  fraction kept: 100.0
################### 

################### 
Applying jet selection
 jet pT > 5 GeV 40158  fraction kept: 100.0
################### 



UndefinedVariableError: name 'jet_qt' is not defined

In [20]:
rc('text', usetex=True)

fig = plt.figure(figsize=(8,6))


df_cut = df_jet['NC'].query('e_pt > %2.2f and e_pt<%2.2f'%(15,20))
printSummary(df_cut)
print '###########'
    
plt.hist(df_cut['jet_qtnormjetpt'],bins=10,range=(0,1.05),normed=True,histtype='step')    
plt.xlabel(r'$q_{T}/p_{\mathrm{T}}^{jet}$', fontsize=25)
plt.ylabel(r'$1/\sigma_{\mathrm{DIS}} \times d\sigma/dq_{T}$ $\ \mathrm{ GeV}^{-1}$ ', fontsize=25)
#plt.tick_params('both',labelsize=20)

plt.text(0.45,1.5, r' $ %2.0f< p_{\mathrm{T}}^{e} < %2.0f$'%(15,20) + '$\ \mathrm{ GeV}$ \n' +
                      r'$\langle x \rangle = %2.2f, \langle Q^{2} \rangle = %2.0f \ \mathrm{GeV}^{2}$'%(df_cut['x'].mean(), df_cut['Q2'].mean()), fontsize=21)

plt.legend(loc='best',frameon=False)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.savefig('qtplot.png', bbox_inches='tight')
plt.savefig('qtplot.pdf', bbox_inches='tight')

17.070654  mean lepton pt
0.28188187  mean inelasticity
33.250675  mean jet p
16.494143  mean jet pT
count    4650.000000
mean       16.494143
std         2.618844
min         5.016327
25%        15.360406
50%        16.598737
75%        18.100995
max        27.972452
Name: jet_pt, dtype: float64
0.16644976  mean x
446.62827  mean Q2
count    4650.000000
mean      446.628265
std       200.466064
min       252.260971
25%       328.270294
50%       394.245056
75%       484.457306
max      2107.036377
Name: Q2, dtype: float64
###########


KeyError: 'jet_qtnormjetpt'